In [3]:
# libraries
import pandas as pd




In [111]:
def scrape_season_fixtures(num_seasons):
    """
    Scrapes La Liga fixtures data for multiple seasons and performs preprocessing.

    Changes made to the original data:
    1. Dropping columns "Notes", "Match Report", and "Venue".
    2. Dropping rows with more than 12 NaN values.
    3. Dropping rows with NaN values in the "Score" column.
    4. Renaming columns "xG" to "xG_home" and "xG.1" to "xG_away".
    5. Splitting the "Score" column into "goals_h" (home team goals) and "goals_a" (away team goals).
    6. Categorizing the "Time" column into three groups: 0 for time between 00:00 to 15:59,
       1 for time between 16:00 to 19:59, and 2 for time between 20:00 to 23:59.

    Args:
    - num_seasons (int): Number of seasons to scrape data for.

    Returns:
    - df_combined (DataFrame): Combined DataFrame containing scraped and preprocessed data.
    - team_label_mapping (dict): Dictionary mapping team names to numeric labels.
    """

    current_season = 2024
    url_base = 'https://fbref.com/en/comps/12/'
    df_list = []
    team_label_mapping = {}  
    label_counter = 0
    home_team_avg_attendance = {}

    for i in range(num_seasons):
        season_end = current_season - i
        season_start = season_end - 1
        url_season = f'{url_base}{season_start}-{season_end}/schedule/{season_start}-{season_end}-La-Liga-Scores-and-Fixtures'
        df = pd.read_html(url_season)[0]
        df['Season'] = f'{season_start}-{season_end}'
        df.drop(columns=["Notes", "Match Report", "Venue"], inplace=True)
        df = df.dropna(thresh=df.shape[1] - 12 + 1)
        df = df.dropna(subset=['Score'])
        df.rename(columns={"xG": "xG_home", "xG.1": "xG_away"}, inplace=True)
        df[['goals_h', 'goals_a']] = df['Score'].str.split('–', expand=True).astype(int)

        # Categorize the 'Time' column
        df['Time'] = pd.to_datetime(df['Time']).dt.hour
        df['Time_group'] = pd.cut(df['Time'], bins=[0, 16, 20, 24], labels=[0, 1, 2], right=False)
        df.drop(columns=['Time'], inplace=True)  # Drop the original 'Time' column
        
        for team in pd.concat([df['Home'], df['Away']]).unique():
            if team not in team_label_mapping:
                team_label_mapping[team] = label_counter
                label_counter += 1
                home_team_avg_attendance[team] = 0
        df_list.append(df)

    df_combined = pd.concat(df_list, ignore_index=True)

    for home_team in home_team_avg_attendance:
        home_team_avg_attendance[home_team] = df_combined[df_combined['Home'] == home_team]['Attendance'].mean().round().astype(int)

    df_combined['Attendance'] = df_combined.apply(lambda row: home_team_avg_attendance[row['Home']] if pd.isna(row['Attendance']) else row['Attendance'], axis=1)

    df_combined['Home'] = df_combined['Home'].map(team_label_mapping)
    df_combined['Away'] = df_combined['Away'].map(team_label_mapping)
    df_combined['target'] = df_combined.apply(lambda x: 0 if x['goals_h'] > x['goals_a'] else (1 if x['goals_a'] > x['goals_h'] else 2), axis=1)

    return df_combined, team_label_mapping

In [16]:
def scrape_season_fixtures(num_seasons):
    """
    Scrapes La Liga fixtures data for multiple seasons and performs preprocessing.

    Changes made to the original data:
    1. Dropping columns "Notes", "Match Report", and "Venue".
    2. Dropping rows with more than 12 NaN values.
    3. Dropping rows with NaN values in the "Score" column.
    4. Renaming columns "xG" to "xG_home" and "xG.1" to "xG_away".
    5. Splitting the "Score" column into "goals_h" (home team goals) and "goals_a" (away team goals).
    6. Categorizing the "Time" column into three groups: 0 for time between 00:00 to 15:59,
       1 for time between 16:00 to 19:59, and 2 for time between 20:00 to 23:59.
    7. Filling NaN values in the "xG_home" and "xG_away" columns with the average xG per team across all seasons.

    Args:
    - num_seasons (int): Number of seasons to scrape data for.

    Returns:
    - df_combined (DataFrame): Combined DataFrame containing scraped and preprocessed data.
    - team_label_mapping (dict): Dictionary mapping team names to numeric labels.
    """

    current_season = 2024
    url_base = 'https://fbref.com/en/comps/12/'
    df_list = []
    team_label_mapping = {}  
    label_counter = 0
    home_team_avg_attendance = {}

    for i in range(num_seasons):
        season_end = current_season - i
        season_start = season_end - 1
        url_season = f'{url_base}{season_start}-{season_end}/schedule/{season_start}-{season_end}-La-Liga-Scores-and-Fixtures'
        df = pd.read_html(url_season)[0]
        df['Season'] = f'{season_start}-{season_end}'
        df.drop(columns=["Notes", "Match Report", "Venue"], inplace=True)
        df = df.dropna(thresh=df.shape[1] - 12 + 1)
        df = df.dropna(subset=['Score'])
        df.rename(columns={"xG": "xG_home", "xG.1": "xG_away"}, inplace=True)
        df[['goals_h', 'goals_a']] = df['Score'].str.split('–', expand=True).astype(int)

        # Categorize the 'Time' column
        df['Time'] = pd.to_datetime(df['Time'], errors='coerce').dt.hour
        df['Time_group'] = pd.cut(df['Time'], bins=[0, 16, 20, 24], labels=[0, 1, 2], right=False)
        df.drop(columns=['Time'], inplace=True)  # Drop the original 'Time' column
        
        for team in pd.concat([df['Home'], df['Away']]).unique():
            if team not in team_label_mapping:
                team_label_mapping[team] = label_counter
                label_counter += 1
                home_team_avg_attendance[team] = 0
        df_list.append(df)

    df_combined = pd.concat(df_list, ignore_index=True)

    


    for home_team in home_team_avg_attendance:
        home_team_avg_attendance[home_team] = df_combined[df_combined['Home'] == home_team]['Attendance'].mean().round().astype(int)

    df_combined['Attendance'] = df_combined.apply(lambda row: home_team_avg_attendance[row['Home']] if pd.isna(row['Attendance']) else row['Attendance'], axis=1)

    df_combined['Home'] = df_combined['Home'].map(team_label_mapping)
    df_combined['Away'] = df_combined['Away'].map(team_label_mapping)
    df_combined['target'] = df_combined.apply(lambda x: 0 if x['goals_h'] > x['goals_a'] else (1 if x['goals_a'] > x['goals_h'] else 2), axis=1)

    
    for i in df_combined['Home'].unique():
        # Filter DataFrame for team i in the "Home" column
        team_home = df_combined[df_combined['Home'] == i]
        
        # Calculate the average of "xG_home" for team i, ignoring NaN values
        avg_xG_home_team = team_home['xG_home'].mean(skipna=True)
        
        # Filter DataFrame for team i in the "Away" column
        team_away = df_combined[df_combined['Away'] == i]
    
        # Calculate the average of "xG_away" for team i, ignoring NaN values
        avg_xG_away_team = team_away['xG_away'].mean(skipna=True)
    
        # Fill NaN values in 'xG_home' and 'xG_away' columns for team i
        df_combined.loc[df_combined['Home'] == i, 'xG_home'] = df_combined.loc[df_combined['Home'] == i, 'xG_home'].fillna(avg_xG_home_team)
        df_combined.loc[df_combined['Away'] == i, 'xG_away'] = df_combined.loc[df_combined['Away'] == i, 'xG_away'].fillna(avg_xG_away_team)

        nan_condition = (df_combined['Home'] == i) & (df_combined['xG_home'].isna()) & (df_combined['Away'] == i) & (df_combined['xG_away'].isna())
        df_combined.loc[nan_condition, ['xG_home', 'xG_away']] = 0.0

        
    
    return df_combined, team_label_mapping

In [22]:
def scrape_future_fixtures(wk, current_season):
    """
    Scrapes La Liga fixtures data for a specific week and current season.

    Args:
    - wk (int): Week number to scrape data for.
    - current_season (int): Current season (year) to scrape data for.

    Returns:
    - df_future (DataFrame): DataFrame containing scraped data for future fixtures.
    """

    url_base = 'https://fbref.com/en/comps/12/'
    url_week = f'{url_base}{current_season - 1}-{current_season}/schedule/La-Liga-Scores-and-Fixtures'
    df_week = pd.read_html(url_week)[0]

    # Filter DataFrame for the specified week and current season
    season_end = current_season
    season_start = season_end - 1
    df_week['Season'] = f'{season_start}-{season_end}'
    df_future = df_week[(df_week['Wk'] == wk) & (df_week['Season'] == f'{current_season - 1}-{current_season}')]

    return df_future

In [2]:
url_df = 'https://fbref.com/en/comps/12/2022-2023/schedule/2022-2023-La-Liga-Scores-and-Fixtures'

In [3]:
df = pd.read_html(url_df)
df

[       Wk  Day        Date   Time        Home   xG Score  xG.1  \
 0     1.0  Fri  2022-08-12  21:00     Osasuna  1.5   2–1   0.9   
 1     1.0  Sat  2022-08-13  17:00  Celta Vigo  0.4   2–2   1.1   
 2     1.0  Sat  2022-08-13  19:00  Valladolid  1.0   0–3   1.5   
 3     1.0  Sat  2022-08-13  21:00   Barcelona  1.9   0–0   0.5   
 4     1.0  Sun  2022-08-14  17:30       Cádiz  0.2   0–1   1.7   
 ..    ...  ...         ...    ...         ...  ...   ...   ...   
 413  38.0  Sun  2023-06-04  21:00  Valladolid  0.9   0–0   0.5   
 414  38.0  Sun  2023-06-04  21:00       Betis  0.6   1–1   1.2   
 415  38.0  Sun  2023-06-04  21:00       Elche  1.6   1–1   2.5   
 416  38.0  Sun  2023-06-04  21:00  Celta Vigo  0.7   2–1   1.2   
 417  38.0  Sun  2023-06-04  21:00    Espanyol  0.6   3–3   1.8   
 
                Away  Attendance                            Venue  \
 0           Sevilla     18536.0                 Estadio El Sadar   
 1          Espanyol     13859.0              Estadio de

In [5]:
df = pd.read_html(url_df)[0]

In [6]:
df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2022-08-12,21:00,Osasuna,1.5,2–1,0.9,Sevilla,18536.0,Estadio El Sadar,Carlos del Cerro,Match Report,NaN
1,1.0,Sat,2022-08-13,17:00,Celta Vigo,0.4,2–2,1.1,Espanyol,13859.0,Estadio de Balaídos,Miguel Ángel Ortiz Arias,Match Report,NaN
2,1.0,Sat,2022-08-13,19:00,Valladolid,1.0,0–3,1.5,Villarreal,17543.0,Estadio Municipal José Zorrilla,Mario Melero,Match Report,NaN
3,1.0,Sat,2022-08-13,21:00,Barcelona,1.9,0–0,0.5,Rayo Vallecano,81104.0,Camp Nou,Alejandro Hernández,Match Report,NaN
4,1.0,Sun,2022-08-14,17:30,Cádiz,0.2,0–1,1.7,Real Sociedad,16570.0,Estadio Nuevo Mirandilla,Isidro Díaz de Mera,Match Report,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Wk            380 non-null    float64
 1   Day           380 non-null    object 
 2   Date          380 non-null    object 
 3   Time          380 non-null    object 
 4   Home          380 non-null    object 
 5   xG            380 non-null    float64
 6   Score         380 non-null    object 
 7   xG.1          380 non-null    float64
 8   Away          380 non-null    object 
 9   Attendance    380 non-null    float64
 10  Venue         380 non-null    object 
 11  Referee       380 non-null    object 
 12  Match Report  380 non-null    object 
 13  Notes         0 non-null      float64
dtypes: float64(5), object(9)
memory usage: 45.8+ KB


In [14]:
df.drop(columns=["Notes", "Match Report"], axis =1, inplace = True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Wk          380 non-null    float64
 1   Day         380 non-null    object 
 2   Date        380 non-null    object 
 3   Time        380 non-null    object 
 4   Home        380 non-null    object 
 5   xG          380 non-null    float64
 6   Score       380 non-null    object 
 7   xG.1        380 non-null    float64
 8   Away        380 non-null    object 
 9   Attendance  380 non-null    float64
 10  Venue       380 non-null    object 
 11  Referee     380 non-null    object 
dtypes: float64(4), object(8)
memory usage: 39.3+ KB


In [19]:
df[df['Wk'] == 17]

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee
176,17.0,Fri,2023-01-13,21:00,Celta Vigo,2.1,1–1,0.1,Villarreal,13931.0,Estadio de Balaídos,Alberola Rojas
177,17.0,Sat,2023-01-14,14:00,Valladolid,0.6,0–1,1.1,Rayo Vallecano,19969.0,Estadio Municipal José Zorrilla,Mario Melero
178,17.0,Sat,2023-01-14,16:15,Girona,2.2,2–1,1.1,Sevilla,12035.0,Estadi Municipal de Montilivi,Valentín Pizarro
179,17.0,Sat,2023-01-14,18:30,Osasuna,1.6,1–0,0.5,Mallorca,19887.0,Estadio El Sadar,César Soto
180,17.0,Sat,2023-01-14,21:00,Real Sociedad,2.0,3–1,0.3,Athletic Club,38296.0,Reale Arena,Guillermo Cuadra
181,17.0,Sun,2023-01-15,14:00,Getafe,0.5,1–2,0.8,Espanyol,10286.0,Coliseum Alfonso Pérez,Jorge Figueroa
182,17.0,Sun,2023-01-15,16:15,Almería,1.4,1–1,2.4,Atlético Madrid,14128.0,Power Horse Stadium,José Sánchez
183,17.0,Mon,2023-01-16,21:00,Cádiz,1.6,1–1,0.6,Elche,17389.0,Estadio Nuevo Mirandilla,Carlos del Cerro
207,17.0,Wed,2023-02-01,21:00,Betis,0.6,1–2,1.9,Barcelona,48181.0,Estadio Benito Villamarín,Ricardo de Burgos
208,17.0,Thu,2023-02-02,21:00,Real Madrid,2.3,2–0,0.2,Valencia,51926.0,Estadio Santiago Bernabéu,Alberola Rojas


In [17]:
test_df, team_labels=scrape_season_fixtures(10)

C:\Users\Pedro\AppData\Local\Temp\ipykernel_16736\3533559302.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce').dt.hour
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16736\3533559302.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce').dt.hour
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16736\3533559302.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce').dt.hour
C:\Users\Pedro\AppData\Local\Temp\ipykernel

In [4]:
team_labels

{'Almería': 0,
 'Sevilla': 1,
 'Real Sociedad': 2,
 'Las Palmas': 3,
 'Athletic Club': 4,
 'Celta Vigo': 5,
 'Villarreal': 6,
 'Getafe': 7,
 'Cádiz': 8,
 'Atlético Madrid': 9,
 'Mallorca': 10,
 'Valencia': 11,
 'Osasuna': 12,
 'Girona': 13,
 'Barcelona': 14,
 'Betis': 15,
 'Alavés': 16,
 'Granada': 17,
 'Rayo Vallecano': 18,
 'Real Madrid': 19,
 'Valladolid': 20,
 'Espanyol': 21,
 'Elche': 22,
 'Levante': 23,
 'Eibar': 24,
 'Huesca': 25,
 'Leganés': 26,
 'La Coruña': 27,
 'Málaga': 28,
 'Sporting Gijón': 29,
 'Córdoba': 30}

In [173]:
team_avg_attendance = test_df.groupby('Home')['Attendance'].mean()

for team, label in team_labels.items():
    avg_attendance = team_avg_attendance[label]
    print(f"Average attendance for {team}: {avg_attendance}")

Average attendance for Almería: 12294.134615384615
Average attendance for Sevilla: 33494.67741935484
Average attendance for Real Sociedad: 24915.935135135136
Average attendance for Las Palmas: 20675.083333333332
Average attendance for Athletic Club: 40387.215053763444
Average attendance for Celta Vigo: 16434.221621621622
Average attendance for Villarreal: 16612.360215053763
Average attendance for Getafe: 9746.397590361446
Average attendance for Cádiz: 16494.11267605634
Average attendance for Atlético Madrid: 51492.56989247312
Average attendance for Mallorca: 14724.444444444445
Average attendance for Valencia: 37840.37837837838
Average attendance for Osasuna: 17407.345454545455
Average attendance for Girona: 11241.112676056338
Average attendance for Barcelona: 70564.06989247311
Average attendance for Betis: 43861.754491017964
Average attendance for Alavés: 16435.703125
Average attendance for Granada: 15932.875
Average attendance for Rayo Vallecano: 11134.146788990825
Average attendance 

In [7]:
test_df.Day.value_counts()

Day
Sun    1416
Sat    1291
Fri     302
Mon     241
Wed     211
Thu     132
Tue     121
Name: count, dtype: int64

In [100]:
test_df.Time.value_counts()

Time
21:00    670
18:30    521
16:15    395
20:45    269
14:00    238
12:00    179
22:00    178
16:00    154
19:00    147
20:00    138
20:30    133
19:30    129
13:00    116
21:30     88
18:15     88
17:00     75
18:00     38
17:30     33
22:05     26
20:15     18
22:15     17
19:45     16
19:15     10
17:15      6
21:15      5
22:30      5
23:00      4
18:45      3
16:45      3
15:15      3
15:00      2
21:05      1
16:30      1
Name: count, dtype: int64

In [9]:
test_df.head(10)

,Wk,Day,Date,Home,xG_home,Score,xG_away,Away,Attendance,Referee,Season,goals_h,goals_a,Time_group,target
0,1.0,Fri,2023-08-11,0,1.4,0–2,2.1,18,14837.0,Alberola Rojas,2023-2024,0,2,1,1
1,1.0,Fri,2023-08-11,1,0.7,1–2,1.1,11,34184.0,José Sánchez,2023-2024,1,2,2,1
2,1.0,Sat,2023-08-12,2,1.0,1–1,0.8,13,30848.0,Francisco Hernández,2023-2024,1,1,1,2
3,1.0,Sat,2023-08-12,3,0.9,1–1,1.6,10,24100.0,Víctor García,2023-2024,1,1,1,2
4,1.0,Sat,2023-08-12,4,0.4,0–2,0.9,19,48927.0,Jesús Gil,2023-2024,0,2,2,1
5,1.0,Sun,2023-08-13,5,1.0,0–2,1.0,12,20269.0,Pablo González,2023-2024,0,2,1,1
6,1.0,Sun,2023-08-13,6,0.9,1–2,1.1,15,17825.0,Miguel Ángel Ortiz Arias,2023-2024,1,2,1,1
7,1.0,Sun,2023-08-13,7,0.6,0–0,1.4,14,13410.0,César Soto,2023-2024,0,0,2,2
8,1.0,Mon,2023-08-14,8,0.7,1–0,0.3,16,19306.0,Isidro Díaz de Mera,2023-2024,1,0,1,0
9,1.0,Mon,2023-08-14,9,2.4,3–1,0.9,17,56164.0,Juan Pulido,2023-2024,3,1,2,0


In [11]:
test_df[test_df["Home"] == 19]

,Wk,Day,Date,Home,xG_home,Score,xG_away,Away,Attendance,Referee,Season,goals_h,goals_a,Time_group,target
33,4.0,Sat,2023-09-02,19,2.800000,2–1,0.400000,7,66747.0,Mario Melero,2023-2024,2,1,1,0
47,5.0,Sun,2023-09-17,19,2.000000,2–1,1.600000,2,70092.0,César Soto,2023-2024,2,1,2,0
62,7.0,Wed,2023-09-27,19,1.700000,2–0,0.700000,3,65017.0,José Luis Munuera,2023-2024,2,0,1,0
81,9.0,Sat,2023-10-07,19,3.200000,4–0,0.500000,12,70864.0,Guillermo Cuadra,2023-2024,4,0,1,0
117,12.0,Sun,2023-11-05,19,2.200000,0–0,0.100000,18,70220.0,Juan Martínez,2023-2024,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634,31.0,Sat,2015-04-11,19,2.046875,3–0,1.051316,24,73965.0,Alejandro Hernández,2014-2015,3,0,1,0
3647,32.0,Sat,2015-04-18,19,2.046875,3–1,0.810526,28,78354.0,Iñaki Bikandi,2014-2015,3,1,2,0
3669,34.0,Wed,2015-04-29,19,2.046875,3–0,1.011765,0,63813.0,Ignacio Iglesias,2014-2015,3,0,2,0
3687,36.0,Sat,2015-05-09,19,2.046875,2–2,1.089844,11,79897.0,Carlos Clos Gómez,2014-2015,2,2,2,2


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714 entries, 0 to 3713
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Wk          3714 non-null   float64 
 1   Day         3714 non-null   object  
 2   Date        3714 non-null   object  
 3   Home        3714 non-null   int64   
 4   xG_home     3657 non-null   float64 
 5   Score       3714 non-null   object  
 6   xG_away     3657 non-null   float64 
 7   Away        3714 non-null   int64   
 8   Attendance  3714 non-null   float64 
 9   Referee     3714 non-null   object  
 10  Season      3714 non-null   object  
 11  goals_h     3714 non-null   int32   
 12  goals_a     3714 non-null   int32   
 13  Time_group  3714 non-null   category
 14  target      3714 non-null   int64   
dtypes: category(1), float64(4), int32(2), int64(3), object(5)
memory usage: 381.1+ KB


In [23]:
test_df.target.value_counts()

target
0    1694
1    1054
2     970
Name: count, dtype: int64

In [108]:
test_df["Referee"].nunique()


48

In [17]:
test_df[(test_df["Wk"]== 30) & (test_df["Home"]==14)]

,Wk,Day,Date,Home,xG_home,Score,xG_away,Away,Attendance,Referee,Season,goals_h,goals_a,Time_group,target
293,30.0,Sat,2024-03-30,14,1.700000,1–0,0.300000,3,46788.0,Mateo Busquets,2023-2024,1,0,2,0
591,30.0,Sun,2023-04-23,14,1.800000,1–0,1.300000,9,80965.0,José Sánchez,2022-2023,1,0,1,0
971,30.0,Sun,2022-04-03,14,1.900000,1–0,0.600000,1,76112.0,José Sánchez,2021-2022,1,0,2,0
2866,30.0,Wed,2017-04-05,14,2.181538,3–0,1.175194,1,85511.0,Carlos Clos,2016-2017,3,0,1,0
3628,30.0,Wed,2015-04-08,14,2.181538,4–0,1.011765,0,73837.0,Alejandro Hernández,2014-2015,4,0,2,0


In [126]:
# Ensure that team labels are mapped to team names correctly
team_labels_names = {v: k for k, v in team_labels.items()}

# Iterate over each season and print the average xG for each team
for season in test_df['Season'].unique():
    print(f"Season: {season}")
    for team_label, team_name in team_labels_names.items():
        # Check if the team_label exists in average_xG_per_team_season_home and average_xG_per_team_season_away
        if team_label in average_xG_per_team_season_home[season].index:
            avg_xG_home = average_xG_per_team_season_home[season][team_label]
        else:
            avg_xG_home = 0.0
            
        if team_label in average_xG_per_team_season_away[season].index:
            avg_xG_away = average_xG_per_team_season_away[season][team_label]
        else:
            avg_xG_away = 0.0
            
        if avg_xG_home != 0.0 or avg_xG_away != 0.0:  # Exclude teams with xG = 0.00
            print(f"Team: {team_name}, Average xG (Home): {avg_xG_home:.2f}, Average xG (Away): {avg_xG_away:.2f}")
    print()

Season: 2023-2024
Team: Almería, Average xG (Home): 1.51, Average xG (Away): 1.01
Team: Sevilla, Average xG (Home): 1.38, Average xG (Away): 0.84
Team: Real Sociedad, Average xG (Home): 1.26, Average xG (Away): 1.29
Team: Las Palmas, Average xG (Home): 0.95, Average xG (Away): 0.79
Team: Athletic Club, Average xG (Home): 1.97, Average xG (Away): 1.06
Team: Celta Vigo, Average xG (Home): 1.42, Average xG (Away): 1.07
Team: Villarreal, Average xG (Home): 1.69, Average xG (Away): 1.17
Team: Getafe, Average xG (Home): 1.21, Average xG (Away): 1.19
Team: Cádiz, Average xG (Home): 1.09, Average xG (Away): 0.65
Team: Atlético Madrid, Average xG (Home): 2.09, Average xG (Away): 1.32
Team: Mallorca, Average xG (Home): 0.91, Average xG (Away): 1.29
Team: Valencia, Average xG (Home): 1.19, Average xG (Away): 0.92
Team: Osasuna, Average xG (Home): 1.21, Average xG (Away): 0.94
Team: Girona, Average xG (Home): 1.69, Average xG (Away): 1.73
Team: Barcelona, Average xG (Home): 2.36, Average xG (Away)

In [6]:
# Ensure that team labels are mapped to team names correctly
team_labels_names = {v: k for k, v in team_labels.items()}

# Iterate over each season and print the average xG for each team
for season in test_df['Season'].unique():
    print(f"Season: {season}")
    for team_label, team_name in team_labels_names.items():
        # Check if the team_label exists in average_xG_per_team_season_home and average_xG_per_team_season_away
        if team_label in average_xG_per_team_season_home[season].index:
            avg_xG_home = average_xG_per_team_season_home[season][team_label]
        else:
            avg_xG_home = 0.0
            
        if team_label in average_xG_per_team_season_away[season].index:
            avg_xG_away = average_xG_per_team_season_away[season][team_label]
        else:
            avg_xG_away = 0.0
            
        if avg_xG_home != 0.0 or avg_xG_away != 0.0:  # Exclude teams with xG = 0.00
            print(f"Team: {team_name}, Average xG (Home): {avg_xG_home:.2f}, Average xG (Away): {avg_xG_away:.2f}")
    print()

Season: 2023-2024


NameError: name 'average_xG_per_team_season_home' is not defined

In [162]:
type(average_xG_per_team_season_home)

pandas.core.series.Series

In [166]:
# Filter DataFrame for team 0 in the "Home" column
team_home = df[df['Home'] == i]

# Calculate the average of "xG_home" for team 0, ignoring NaN values
avg_xG_home_team = team_home['xG_home'].mean(skipna=True)

# Filter DataFrame for team 0 in the "Away" column
team_away = df[df['Away'] == 0]

# Calculate the average of "xG_away" for team 0, ignoring NaN values
avg_xG_away_team = team_away['xG_away'].mean(skipna=True)

print("Average xG_home for Team 0:", avg_xG_home_team_0)
print("Average xG_away for Team 0:", avg_xG_away_team_0)

Average xG_home for Team 0: 1.4303030303030309
Average xG_away for Team 0: 1.0117647058823533


In [177]:
for i in test_df['Home'].unique():
    # Filter DataFrame for team i in the "Home" column
    team_home = test_df[test_df['Home'] == i]
    
    # Calculate the average of "xG_home" for team i, ignoring NaN values
    avg_xG_home_team = team_home['xG_home'].mean(skipna=True)
    
    # Filter DataFrame for team i in the "Away" column
    team_away = test_df[test_df['Away'] == i]

    # Calculate the average of "xG_away" for team i, ignoring NaN values
    avg_xG_away_team = team_away['xG_away'].mean(skipna=True)

    # Fill NaN values in 'xG_home' and 'xG_away' columns for team i
    test_df.loc[test_df['Home'] == i, 'xG_home'] = test_df.loc[test_df['Home'] == i, 'xG_home'].fillna(avg_xG_home_team)
    test_df.loc[test_df['Away'] == i, 'xG_away'] = test_df.loc[test_df['Away'] == i, 'xG_away'].fillna(avg_xG_away_team)

    print(i)
    print(test_df.loc[test_df['Home'] == i, 'xG_home'])
    print(test_df.loc[test_df['Away'] == i, 'xG_away'])


0
0       1.400000
13      1.300000
31      3.000000
53      2.600000
74      2.900000
100     0.300000
121     0.400000
145     2.400000
164     0.300000
195     1.700000
207     1.400000
239     0.900000
253     0.500000
278     2.000000
295     0.700000
313     2.300000
338     0.200000
360     1.200000
386     1.800000
400     2.300000
422     0.500000
444     0.600000
455     1.400000
467     2.100000
490     2.300000
516     1.000000
521     0.400000
540     2.400000
566     0.900000
581     0.800000
609     1.700000
633     1.800000
654     1.700000
3332    1.430303
3349    1.430303
3373    1.430303
3394    1.430303
3410    1.430303
3430    1.430303
3458    1.430303
3469    1.430303
3503    1.430303
3534    1.430303
3549    1.430303
3572    1.430303
3594    1.430303
3612    1.430303
3632    1.430303
3655    1.430303
3678    1.430303
3686    1.430303
3705    1.430303
Name: xG_home, dtype: float64
22      0.300000
45      1.500000
59      1.800000
79      0.100000
95      1.000000

In [191]:
    for i in df_combined['Home'].unique():
        # Filter DataFrame for team i in the "Home" column
        team_home = df_combined[df_combined['Home'] == i]
        
        # Calculate the average of "xG_home" for team i, ignoring NaN values
        avg_xG_home_team = team_home['xG_home'].mean(skipna=True)
        
        # Filter DataFrame for team i in the "Away" column
        team_away = df_combined[df_combined['Away'] == i]
    
        # Calculate the average of "xG_away" for team i, ignoring NaN values
        avg_xG_away_team = team_away['xG_away'].mean(skipna=True)
    
        # Fill NaN values in 'xG_home' and 'xG_away' columns for team i
        df_combined.loc[df_combined['Home'] == i, 'xG_home'] = df_combined.loc[df_combined['Home'] == i, 'xG_home'].fillna(avg_xG_home_team)
        df_combined.loc[df_combined['Away'] == i, 'xG_away'] = df_combined.loc[df_combined['Away'] == i, 'xG_away'].fillna(avg_xG_away_team)

        nan_condition = (df_combined['Home'] == i) & (df_combined['xG_home'].isna()) & (df_combined['Away'] == i) & (df_combined['xG_away'].isna())
        df_combined.loc[nan_condition, ['xG_home', 'xG_away']] = 0.0

,Wk,Day,Date,Home,xG_home,Score,xG_away,Away,Attendance,Referee,Season,goals_h,goals_a,Time_group,target
0,1.0,Fri,2023-08-11,0,1.400000,0–2,2.100000,18,14837.0,Alberola Rojas,2023-2024,0,2,1,1
13,2.0,Sat,2023-08-19,0,1.300000,1–3,2.000000,19,17561.0,José Sánchez,2023-2024,1,3,1,1
31,4.0,Fri,2023-09-01,0,3.000000,2–3,1.500000,5,14196.0,Ricardo de Burgos,2023-2024,2,3,2,1
53,6.0,Sat,2023-09-23,0,2.600000,2–2,0.800000,11,14266.0,Mateo Busquets,2023-2024,2,2,2,2
74,8.0,Sun,2023-10-01,0,2.900000,3–3,2.900000,17,14264.0,Pablo González,2023-2024,3,3,0,2
100,11.0,Sat,2023-10-28,0,0.300000,1–2,0.900000,3,12132.0,Francisco Hernández,2023-2024,1,2,0,1
121,13.0,Sat,2023-11-11,0,0.400000,1–3,2.400000,2,13016.0,Isidro Díaz de Mera,2023-2024,1,3,1,1
145,15.0,Sun,2023-12-03,0,2.400000,0–0,0.500000,15,14253.0,Alejandro Muñíz,2023-2024,0,0,1,2
164,17.0,Sun,2023-12-17,0,0.300000,0–0,1.900000,10,11936.0,Pablo González,2023-2024,0,0,0,2
195,20.0,Sun,2024-01-14,0,1.700000,0–0,0.300000,13,12111.0,Mateo Busquets,2023-2024,0,0,0,2


In [192]:
avg_xG_home_team = team_home['xG_home'].mean(skipna=True)
avg_xG_home_team

1.4303030303030309

In [193]:
test_df.loc[test_df['Home'] == 0, 'xG_home'] = test_df.loc[test_df['Home'] == 0, 'xG_home'].fillna(avg_xG_home_team)

In [196]:
test_df.loc[test_df['Home'] == 19, 'xG_home'].info()

<class 'pandas.core.series.Series'>
Index: 185 entries, 33 to 3708
Series name: xG_home
Non-Null Count  Dtype  
--------------  -----  
185 non-null    float64
dtypes: float64(1)
memory usage: 2.9 KB


In [197]:
9*19

171

In [199]:
nan_rows = test_df[test_df['xG_home'].isna()]
print(nan_rows)

        Wk  Day        Date  Home  xG_home Score   xG_away  Away  Attendance  \
2574   1.0  Sun  2016-08-21    29      NaN   2–1  1.092188     4     24833.0   
2595   3.0  Sun  2016-09-11    29      NaN   2–1  0.998246    26     22682.0   
2621   6.0  Sat  2016-09-24    29      NaN   0–5  1.704687    14     26098.0   
2646   8.0  Sun  2016-10-16    29      NaN   1–2  1.089844    11     25309.0   
2660  10.0  Sat  2016-10-29    29      NaN   1–1  1.178125     1     22803.0   
2686  12.0  Sun  2016-11-20    29      NaN   1–3  1.187597     2     23439.0   
2706  14.0  Sun  2016-12-04    29      NaN   3–1  0.905556    12     25800.0   
2720  16.0  Sat  2016-12-17    29      NaN   1–3  1.308594     6     19900.0   
2744  18.0  Sun  2017-01-15    29      NaN   2–3  1.051316    24     20050.0   
2773  21.0  Sun  2017-02-05    29      NaN   2–4  0.933636    16     21752.0   
2787  23.0  Sat  2017-02-18    29      NaN   1–4  1.287500     9     24480.0   
2804  24.0  Sun  2017-02-26    29      N

In [208]:
filtered_data = test_df[(test_df['Season'] == '2023-2024') & (test_df['Home'] == 19)]


average_goals_h = filtered_data['goals_h'].mean()

print("Average goals_h for Season 2023-2024, Home team 19:", average_goals_h)

Average goals_h for Season 2023-2024, Home team 19: 2.5


In [209]:
filtered_data_away = test_df[(test_df['Season'] == '2023-2024') & (test_df['Away'] == 4)]

average_goals_a = filtered_data_away['goals_a'].mean()

print("Average goals_a for Season 2023-2024, Away team 4:", average_goals_a)

Average goals_a for Season 2023-2024, Away team 4: 1.0


In [25]:
df_pred=scrape_future_fixtures(30, 2024)

In [26]:
df_pred

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Season
321,30.0,Fri,2024-03-29,21:00,Cádiz,1.6,1–0,0.5,Granada,15422.0,Estadio Nuevo Mirandilla,Guillermo Cuadra,Match Report,NaN,2023-2024
322,30.0,Sat,2024-03-30,14:00,Getafe,1.6,0–1,0.8,Sevilla,10139.0,Coliseum Alfonso Pérez,Javier Villanueva,Match Report,NaN,2023-2024
323,30.0,Sat,2024-03-30,16:15,Almería,0.6,0–3,1.0,Osasuna,10565.0,Power Horse Stadium,Pablo González,Match Report,NaN,2023-2024
324,30.0,Sat,2024-03-30,18:30,Valencia,1.5,0–0,0.8,Mallorca,42567.0,Estadio de Mestalla,Miguel Ángel Ortiz Arias,Match Report,NaN,2023-2024
325,30.0,Sat,2024-03-30,21:00,Barcelona,1.7,1–0,0.3,Las Palmas,46788.0,Estadi Olímpic Lluís Companys,Mateo Busquets,Match Report,NaN,2023-2024
326,30.0,Sun,2024-03-31,14:00,Celta Vigo,NaN,NaN,NaN,Rayo Vallecano,NaN,Estadio de Balaídos,NaN,Head-to-Head,NaN,2023-2024
327,30.0,Sun,2024-03-31,16:15,Girona,NaN,NaN,NaN,Betis,NaN,Estadi Municipal de Montilivi,NaN,Head-to-Head,NaN,2023-2024
328,30.0,Sun,2024-03-31,18:30,Alavés,NaN,NaN,NaN,Real Sociedad,NaN,Estadio de Mendizorroza,NaN,Head-to-Head,NaN,2023-2024
329,30.0,Sun,2024-03-31,21:00,Real Madrid,NaN,NaN,NaN,Athletic Club,NaN,Estadio Santiago Bernabéu,NaN,Head-to-Head,NaN,2023-2024
330,30.0,Mon,2024-04-01,21:00,Villarreal,NaN,NaN,NaN,Atlético Madrid,NaN,Estadio de la Cerámica,NaN,Head-to-Head,NaN,2023-2024


In [12]:
df2 = pd.read_html("https://fbref.com/en/matches/525758cc/Sevilla-Valencia-August-11-2023-La-Liga")[3]

In [13]:
df2

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
               Player                  #             Nation   
0   Youssef En-Nesyri               15.0             ma MAR   
1    Federico Gattoni                2.0             ar ARG   
2         Érik Lamela               17.0             ar ARG   
3       Óliver Torres               21.0             es ESP   
4       Lucas Ocampos                5.0             ar ARG   
5                Suso                7.0             es ESP   
6        Ivan Rakitić               10.0             hr CRO   
7            Fernando               20.0             br BRA   
8         Djibril Sow               18.0             ch SUI   
9        Marcos Acuña               19.0             ar ARG   
10     Nemanja Gudelj                6.0             rs SRB   
11          Loïc Bade               22.0             fr FRA   
12        Jesús Navas               16.0             es ESP   
13     Juanlu Sánchez               26.0             es ESP   
14     Yassine Bounou               13.0             ma MAR   
15         15 Players                NaN                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Performance      \
                  Pos                Age                Min         Gls Ast   
0                  FW             26-071                 81           1   0   
1                  CB             24-176                  9           0   0   
2                  LW             31-160                 55           0   0   
3                  AM             28-274                 35           0   0   
4               RW,LW             29-031                 90           0   0   
5               AM,RW             29-265                 90           0   1   
6                  DM             35-154                 90           0   0   
7                  DM             36-017                 68           0   0   
8                  DM             26-186                 22           0   0   
9                  LB             31-287                 90           0   0   
10                 CB             31-268                 90           0   0   
11                 CB             23-122                 80           0   0   
12                 RB             37-263                 68           0   0   
13                 RB             19-361                 22           0   0   
14                 GK             32-128                 90           0   0   
15                NaN                NaN                980           1   1   

             ... SCA     Passes                 Carries      Take-Ons       
   PK PKatt  ... SCA GCA    Cmp  Att  Cmp% PrgP Carries PrgC      Att Succ  
0   0     0  ...   0   0      3    6  50.0    0       8    1        3    0  
1   0     0  ...   0   0      5    6  83.3    0       2    0        0    0  
2   0     0  ...   1   0      6    7  85.7    0       9    0        1    0  
3   0     0  ...   1   0     13   18  72.2    1      13    1        1    0  
4   0     0  ...   1   0     15   26  57.7    1      28    4        4    0  
5   0     0  ...   4   2     23   39  59.0    3      32    3        3    1  
6   0     0  ...   2   0     46   69  66.7    2      40    3        5    2  
7   0     0  ...   1   0     25   30  83.3    2      24    0        3    1  
8   0     0  ...   0   0      8   10  80.0    1       7    0        0    0  
9   0     0  ...   4   0     49   68  72.1    5      36    2        3    1  
10  0     0  ...   0   0     43   63  68.3    4      44    0        0    0  
11  0     0  ...   0   0     40   46  87.0    2      28    0        0    0  
12  0     0  ...   2   0     32   42  76.2    2      25    0        1    0  
13  0     0  ...   1   0     11   13  84.6    0       8    1        2    2  
14  0     0  ...   0   0     20   31  64.5    0      16    0        0    0  
15  0     0  ...  17   2    339  474  71.5   23     320   15       26    7  

[16 rows x 31 columns]